In [1]:
from numba import jit
import numpy as np

In [2]:
def LU(A):
    """Returns the LU-decomposition of a square matrix A."""
    m,n = A.shape
    U = np.copy(A).astype(float)
    L = np.zeros((n,n))
    for i in range(n):
        L[i,i] = 1
    for i in range(1,n):
        for j in range(i):
            L[i,j] = U[i,j]/U[j,j]
            for k in range(j,n):
                U[i,k] -= U[j,k]*U[i,j]/U[j,j]
    return L,U  

In [3]:
def LU_opt(A):
    """Returns the LU-decomposition of a square matrix A."""
    m,n = A.shape
    U = np.copy(A).astype(float)
    L = np.eye(n)
    for i in range(1,n):
        for j in range(i):
            L[i,j] = U[i,j]/U[j,j]
            U[i,j:] -= L[i,j]*U[j,j:]
    return L,U  

In [4]:
n = 100
A = np.random.random_integers(1,9,n**2).reshape((n,n))

In [5]:
%load_ext Cython

In [6]:
%%cython
import numpy as np
def cy_LU(A):
    """Returns the LU-decomposition of a square matrix A."""
    cdef:
        int m
        int n
        int i
        int j
        int k
        double[:,:] U
        double[:,:] L
    
    m,n = A.shape
    U = np.copy(A).astype(float)
    L = np.zeros((n,n))
    for i in range(n):
        L[i,i] = 1
    for i in range(1,n):
        for j in range(i):
            L[i,j] = U[i,j]/U[j,j]
            for k in range(j,n):
                U[i,k] -= U[j,k]*U[i,j]/U[j,j]
    return L,U  

In [7]:
%%cython
import numpy as np

def cy_LU_opt(A):
    """Returns the LU-decomposition of a square matrix A."""
    cdef:
        int m
        int n
        int i 
        int j
        double[:,:] U
        double[:,:] L
    m,n = A.shape
    U = np.copy(A).astype(float)
    L = np.eye(n)
    for i in xrange(1,n):
        for j in xrange(i):
            L[i,j] = U[i,j]/U[j,j]
            for k in range(j,n):
                U[i,k] -= U[j,k]*L[i,j]
    return L,U  

In [8]:
@jit
def numba_LU_opt(A):
    """Returns the LU-decomposition of a square matrix A."""
    m,n = A.shape
    U = np.copy(A).astype(float)
    L = np.eye(n)
    for i in range(1,n):
        for j in range(i):
            L[i,j] = U[i,j]/U[j,j]
            U[i,j:] -= L[i,j]*U[j,j:]
    return L,U  

In [15]:
@jit
def numba_LU_opt2(A):
    """Returns the LU-decomposition of a square matrix A."""
    m,n = A.shape
    U = np.copy(A).astype(float)
    L = np.eye(n)
    for i in xrange(1,n):
        for j in xrange(i):
            L[i,j] = U[i,j]/U[j,j]
            for k in range(j,n):
                U[i,k] -= U[j,k]*L[i,j]
    return L,U  

In [10]:
%time l,u = LU(A)

CPU times: user 220 ms, sys: 16 ms, total: 236 ms
Wall time: 231 ms


In [11]:
%time l,u = LU_opt(A)

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 12.5 ms


In [12]:
%time l,u = cy_LU(A)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.16 ms


In [13]:
%time l,u = cy_LU_opt(A)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 759 µs


In [14]:
%time l,u = numba_LU_opt(A)

CPU times: user 284 ms, sys: 8 ms, total: 292 ms
Wall time: 522 ms


In [16]:
%time l,u = numba_LU_opt2(A)

CPU times: user 240 ms, sys: 4 ms, total: 244 ms
Wall time: 239 ms


In [19]:
def pysum(A):
    total = 0
    for i in xrange(len(A)):
        total += A[i]
    return total

In [21]:
%%cython
def cysum(double[:] A):
    cdef double total = 0
    cdef int i
    for i in xrange(len(A)):
        total += A[i]
    return total

In [68]:
@jit
def numba_sum(A):
    total = 0
    for i in xrange(len(A)):
        total += A[i]
    return total

In [39]:
A = np.random.rand(1000000)

In [49]:
%timeit pysum(A)

10 loops, best of 3: 136 ms per loop


In [50]:
%timeit cysum(A)

1000 loops, best of 3: 953 µs per loop


In [69]:
times = []
for i in xrange(100):
    A = np.random.rand(1000000)
    before = time.time()
    a = numba_sum(A)
    after = time.time()
    times.append(after-before)

In [66]:
import time

In [70]:
max(times)

0.04695391654968262

In [71]:
min(times)

0.0009210109710693359

In [72]:
np.average(times)

0.001398007869720459

In [5]:
import numpy as np

In [6]:
def init_tridiag(n):
    """Initializes a random nxn tridiagonal matrix A.
    
    Inputs:
        n (int) : size of array
    
    Returns:
        a (1-D array) : (-1)-th diagonal of A
        b (1-D array) : main diagonal of A
        c (1-D array) : (1)-th diagonal of A
        A (2-D array) : nxn tridiagonal matrix defined by a,b,c. 
    """
    a = np.random.random_integers(-9,9,n).astype("float")
    b = np.random.random_integers(-9,9,n).astype("float")
    c = np.random.random_integers(-9,9,n).astype("float")

    tempA = spar.spdiags([c,b,a],[-1,0,1],n,n)
    A = tempA.todense().T
    return a,b,c,A

In [8]:
def init_tridiag_new(n):
    """Initializes a random nxn tridiagonal matrix A.
    
    Inputs:
        n (int) : size of array
    
    Returns:
        a (1-D array) : (-1)-th diagonal of A
        b (1-D array) : main diagonal of A
        c (1-D array) : (1)-th diagonal of A
        A (2-D array) : nxn tridiagonal matrix defined by a,b,c. 
    """
    a = np.random.random_integers(-9,9,n).astype("float")
    b = np.random.random_integers(-9,9,n).astype("float")
    c = np.random.random_integers(-9,9,n).astype("float")

    A = np.zeros((b.size,b.size))
    np.fill_diagonal(A,b)
    np.fill_diagonal(A[1:,:-1],a[1:])
    np.fill_diagonal(A[:-1,1:],c)
    return a,b,c,A

In [15]:
%time init_tridiag(1000)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.24 ms


(array([-1., -3., -7., -7., -2.,  4.,  4., -1.,  9., -3., -4.,  4., -2.,
         3.,  6., -2.,  6.,  5., -4.,  2.,  0.,  3.,  5., -4.,  3., -2.,
         3., -6.,  8.,  5.,  9., -2., -4., -2., -4., -6.,  7.,  1.,  5.,
        -7.,  9., -8., -7., -4.,  1., -3., -8., -3.,  6.,  3., -6., -5.,
        -3., -8., -8., -3., -5.,  9., -9., -2., -2., -3., -8.,  6.,  1.,
        -4., -3.,  6.,  2., -4., -9.,  9.,  1.,  5., -8.,  6.,  3.,  8.,
        -1.,  8.,  0., -1.,  5., -7.,  1.,  9., -4.,  4.,  3.,  9., -3.,
        -2., -2., -9., -1., -3., -3., -7., -5., -9.,  3., -2.,  6., -8.,
        -6., -9., -1., -6.,  3., -5.,  3., -8.,  1.,  7.,  0.,  2., -5.,
        -6.,  0., -5., -9.,  6., -6., -1.,  8.,  7.,  0.,  4., -8.,  8.,
        -7., -7., -7., -8.,  8., -7.,  0.,  2., -5., -1.,  8.,  0.,  4.,
         3., -1.,  5.,  4.,  8., -3., -7., -2., -6.,  5., -7.,  5., -2.,
        -4., -1.,  8., -1., -3.,  6.,  2., -4.,  3., -5.,  3.,  1., -2.,
         2., -7., -3.,  1.,  9., -9., -3., -2., -4.

In [12]:
import scipy.sparse as spar

In [16]:
%time init_tridiag_new(1000)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3 ms


(array([ 8., -2.,  6., -6., -9., -4., -1., -7.,  2., -2., -5., -1.,  0.,
         4.,  2., -3.,  8., -8., -6.,  0., -9., -4.,  7., -1., -9.,  2.,
         9., -9.,  3., -2.,  9., -3., -8.,  4.,  8., -4.,  6., -3., -8.,
        -6., -2.,  7., -5., -3., -8.,  2.,  6., -9.,  5., -1.,  2., -1.,
         4.,  7.,  6., -9., -2.,  0., -1.,  8., -2.,  3.,  8., -3.,  8.,
         1., -6.,  4., -5.,  4., -3., -1.,  7., -2.,  2.,  4.,  1., -7.,
        -2., -2., -1.,  1.,  3.,  0.,  9.,  3.,  8.,  1., -6.,  1., -3.,
         3.,  5., -1.,  8.,  0., -8.,  6., -6., -1.,  2.,  7., -1., -8.,
        -6., -2.,  8., -7.,  0.,  4., -1.,  7., -4.,  1.,  5.,  5.,  1.,
         5., -2.,  2., -3.,  4., -3.,  7.,  9., -3., -9.,  7., -4.,  3.,
         1., -7., -1., -1., -5., -4.,  5., -7.,  0.,  6., -1.,  9.,  0.,
        -3.,  2., -3.,  3.,  1.,  7.,  5., -9., -1.,  0.,  0.,  3.,  3.,
        -3., -9., -6.,  2., -1.,  0.,  7.,  6.,  5.,  3.,  6., -9.,  2.,
        -9.,  8.,  7., -2., -4.,  6., -8., -4.,  4.

In [17]:
def LU(A):
    """Returns the LU decomposition of a square matrix."""
    U = np.array(np.copy(A), dtype=float)
    L = np.identity(np.sqrt(A.size))
    
    for i in xrange(1,A.shape[0]):
        for j in xrange(i):
            L[i,j] = U[i,j]/U[j,j]
            U[i,j:] -= L[i,j]*U[j,j:]
    return L,U

In [18]:
A = np.random.random_integers(-9,9,25).reshape((5,5))
L,U = LU(A)
np.allclose(L.dot(U),A)

True